# Memory Management in NumPy

ُhe very core of NumPy is the ndarray object. We are going to finish the last important attribute of ndarray: strides, which will give you the full picture of memory layout. Also, it's time to show you that NumPy arrays can deal not only with numbers but also with various types of data; we will talk about record arrays and date time arrays. Lastly, we will show how to read/write NumPy arrays from/to files, and start to do some real-world analysis using NumPy.

## Internal memory layout of an ndarray

An instance of class `ndarray` consists of a contiguous one-dimensional segment of computer memory (owned by the array, or by some other object), combined with an indexing scheme that maps `N` integers into the location of an item in the block. The ranges in which the indices can vary is specified by the **shape** of the array. How many bytes each item takes and how the bytes are interpreted is defined by the data-type object associated with the array.

A particularly interesting attribute of the ndarray object is `flags`. Type the following code:

In [1]:
import numpy as np

In [2]:
x = np.array([1, 2, 3])

In [3]:
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

The `flags` attribute holds information about the memory layout of the array. The `C_CONTIGUOUS` field in the output indicates whether the array was a C-style array. This means that the indexing of this array is done like a C array. This is also called row-major indexing in the case of 2D arrays. This means that, when moving through the array, the row index is incremented first, and then the column index is incremented. In the case of a multidimensional C-style array, the last dimension is incremented first, followed by the last but one, and so on.

Similarly, the `F_CONTIGUOUS` attribute indicates whether the array is a Fortran-style array. Such an array is said to have column-major indexing (R, Julia, and MATLAB use column-major arrays). This means that, when moving through the array, the first index (along the column) is incremented first.

In [5]:
c_array = np.random.rand(10000, 10000) 

In [23]:
f_array = np.asfortranarray(c_array) 

In [24]:
def sum_row(x):
    '''
    Given an array `x`, return the sum of its zeroth row.
    '''
    return np.sum(x[0, :])

In [25]:
def sum_col(x):
    '''
    Given an array `x`, return the sum of its zeroth column.
    '''
    return np.sum(x[:, 0])

Now, let's test the performance of the two functions on both the arrays using IPython's %timeit magic function:

In [26]:
%timeit sum_row(c_array) 

6.03 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%timeit sum_row(f_array) 

101 µs ± 3.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%timeit sum_col(c_array) 

103 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [29]:
%timeit sum_col(f_array) 

6.14 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


As we can see, summing up the row of a C array is much faster than summing up its column. This is because, in a C array, elements in a row are laid out in successive memory locations. The opposite is true for a Fortran array, where the elements of a column are laid out in consecutive memory locations.

This is an important distinction and allows you to suitably arrange your data in an array, depending on the kind of algorithm or operation you are performing. Knowing this distinction can help you speed up your code by orders of magnitude.

## Views and Copies

There are primarily two ways of accessing data by slicing and indexing. They are called copies and views: you can either access elements directly from an array, or create a copy of the array that contains only the accessed elements. Since a view is a reference of the original array (in Python, all variables are references), modifying a view modifies the original array too. This is not true for copies.

The `may_share_memory` function in NumPy miscellaneous routines can be used to determine whether two arrays are copies or views of each other. While this method does the job in most cases, it is not always reliable, since it uses heuristics. It may return incorrect results too. For introductory purposes, however, we shall take it for granted.

Generally, slicing an array creates a view and indexing it creates a copy. Let us study these differences through a few code snippets. First, let's create a random `100x10` array.

In [8]:
x = np.random.rand(100, 10)

Now, let us extract the first five rows of the array and assign them to variable `y`.

In [9]:
y = x[:5, :]

Let us see if `y` is a view of `x`.

In [10]:
np.may_share_memory(x, y)

True

Now let us modify the array `y` and see how it affects `x`. Set all the elements of `y` to zero:

In [52]:
y[:] = 0 

In [53]:
x[:5, :]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

The code snippet prints out five rows of zeros. This is because `y` was just a view, a reference to `x`.

Next, let's create a copy to see the difference. We use the preceding method that uses a random function to create the `x` array, but this time we initialize the `y` array using `numpy.empty` to create an empty array first and then copy the values from `x` to `y`. So, now `y` is not a view/reference of `x` anymore; it's an independent array but has the same values as part of `x`. Let's use the may_share_memory function again to verify that `y` is the copy of `x`:

In [58]:
x = np.random.rand(100, 10)
y = np.empty([5, 10]) 

In [59]:
y[:] = x[:5, :]

In [60]:
np.may_share_memory(x, y)

False

Let's alter the value in `y` and check whether the value of `x` changes as well:

In [61]:
y[:] = 0

In [63]:
x[:5, :]

array([[0.22594285, 0.84943588, 0.63799672, 0.90865088, 0.89087029,
        0.66289172, 0.41134246, 0.46546459, 0.46927432, 0.34504615],
       [0.56702582, 0.04641427, 0.9855564 , 0.40668092, 0.0628503 ,
        0.56129959, 0.35503778, 0.22145278, 0.72152542, 0.70329602],
       [0.26986364, 0.4927989 , 0.4388591 , 0.15837464, 0.65150918,
        0.62545448, 0.47597151, 0.28765198, 0.38625864, 0.56950966],
       [0.08818207, 0.9425105 , 0.59893522, 0.61628436, 0.51038646,
        0.05008038, 0.85750826, 0.55464599, 0.14657257, 0.56587341],
       [0.63737814, 0.30478605, 0.69243957, 0.54875134, 0.55246097,
        0.2616949 , 0.26626916, 0.98765586, 0.80260403, 0.03282723]])

You should see the preceding snippet print out five rows of random numbers as we initialized `x`, so changing `y` to `0` didn't affect `x`.

## Introducing strides

Strides are the indexing scheme in NumPy arrays, and indicate the number of bytes to jump to find the next element. We all know the performance improvements of NumPy come from a homogeneous multidimensional array object with fixed-size items, the `numpy.ndarray` object. We've talked about the `shape` (dimension) of the `ndarray` object, the data type, and the order (the C-style row-major indexing arrays and the Fortran style column-major arrays.) Now it's time to take a closer look at **strides**.

###  Example

Let's start by creating a NumPy array and changing its shape to see the differences in the strides.

In [11]:
x = np.arange(8, dtype = np.int8)

In [12]:
x

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int8)

In [13]:
x.strides

(1,)

In [79]:
x.data

A one-dimensional array `x` is created and its data type is NumPy integer 8, which means each element in the array is an 8-bit integer (1 byte each, and a total of 8 bytes). The strides represent the `tuple` of bytes to step in each dimension when traversing an array. In the previous example, it's one dimension, so we obtain the tuple as `(1, )`. Each element is 1 byte apart from its previous element. When we print out `x.data`, we can get the Python buffer object pointing to the start of the data

Change the shape and see the stride change:

In [14]:
x.shape = 2, 4

In [17]:
x.strides

(4, 1)

Now we change the dimensions of `x` to `2x4` and check the strides again. We can see it changes to `(4, 1)`, which means the elements in the first dimension are four bytes apart, and the array need to jump four bytes to find the next row, but the elements in the second dimension are still 1 byte apart, jumping one byte to find the next column. Let's print out `x.data` again, and we can see that the memory layout of the data remains the same, but only the strides change. The same behavior occurs when we change the shape to be three dimensional: `1x4x2` arrays. (What if our arrays are constructed by the Fortran style order? How will the strides change due to changing the shapes? Try to create a column-major array and do the same exercise to check this out.)

In [18]:
x.shape = 1, 4, 2

In [19]:
x.strides

(8, 2, 1)

In [20]:
x.data

### How can the stride improve our NumPy experience?

So now we know what a stride is, and its relationship to an ndarray object, but how can the stride improve our NumPy experience? Let's do some stride manipulation to get a better sense of this: two arrays are with same content but have different strides:

In [25]:
x = np.ones((10000, )) 

In [26]:
y = np.ones((10000 * 100, ))[::100] 

In [23]:
x.shape, y.shape

((10000,), (10000,))

In [24]:
(x == y).all()

True

We create two NumPy Arrays, `x` and `y`, and do a comparison; we can see that the two arrays are equal. They have the same shape and all the elements are one, but actually the two arrays are different in terms of memory layout. Let's simply use the flags attribute to check the two arrays' memory layout.

In [116]:
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [118]:
y.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

We can see that the `x` array is continuous in both the C and Fortran order while `y` is not. Let's check the strides for the difference:

In [120]:
x.strides, y.strides

((8,), (800,))

Array `x` is created continuously, so in the same dimension each element is eight bytes apart (the default `dtype` of `numpy.ones` is a 64-bit float); however, `y` is created from a subset of 10000 * 100 for every 100 elements, so the index schema in the memory layout is not continuous.

Even though `x` and `y` have the same shape, each element in `y` is 800 bytes apart from each other. When you use NumPy arrays `x` and `y`, you might not notice the difference in indexing, but the memory layout does affect the performance. Let's use the `%timeit` to check this out:

In [121]:
%timeit x.sum()

4.09 µs ± 36.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [122]:
%timeit y.sum()

19.8 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Typically with a fixed cache size, when the stride size gets larger, the hit rate (the fraction of memory accessed that finds data in the cache) will be lower, comparatively, while the miss rate (the fraction of memory accessed that has to go to the memory) will be higher. The cache hit time and miss time compose the average data access time. Let's try to look at our example again from the cache perspective. Array `x` with smaller strides is faster than the larger strides of `y`. The reason for the difference in performance is that the CPU is pulling data from the main memory to its cache in blocks, and the smaller stride means fewer transfers are needed. See the following graph for details, where the red line represents the size of the CPU cache, and blue boxes represent the memory layout containing the data.

It's obvious that if `x` and `y` are both required, 100 blue boxes of data, the required cache time for `x` will be less.

<img src="../images/cpu-cache.jpg" alt="cpu-cache" width=500 align="left" />

### Stride in N-dimensional `ndarray`

A segment of memory is inherently 1-dimensional, and there are many different schemes for arranging the items of an N-dimensional array in a 1-dimensional block. NumPy is flexible, and ndarray objects can accommodate any strided indexing scheme. In a strided scheme, the N-dimensional index (`n_0, n_1, ..., n_{N-1}`) corresponds to the offset (in bytes):

$n_{\text{offset}} = \sum_{k=0}^{N-1} s_k n_k$

from the beginning of the memory block associated with the array. Here, $s_k$ are integers which specify the strides of the array.

In [123]:
import numpy as np

In [124]:
x = np.random.rand(3, 4)

In [125]:
x.itemsize, x.nbytes / x.shape[0]

(8, 32.0)

In [126]:
x.strides

(32, 8)

In [127]:
# .flatten() returns numpy array
x.flatten()

array([0.86715172, 0.75103058, 0.59647724, 0.96017377, 0.70260017,
       0.39786388, 0.03984518, 0.12612843, 0.00936317, 0.49894596,
       0.78386359, 0.43940843])

In [128]:
# .flat returns iterator
np.array(x.flat)

array([0.86715172, 0.75103058, 0.59647724, 0.96017377, 0.70260017,
       0.39786388, 0.03984518, 0.12612843, 0.00936317, 0.49894596,
       0.78386359, 0.43940843])

In [129]:
# a stride of 1 is equal to 32 bytes
x[1]

array([0.70260017, 0.39786388, 0.03984518, 0.12612843])

## Conclusion: Why `np.array` not `list`

In [ ]:
x = [1, '2', [1, 2, 3]]
arr = np.array([1, 2, 3])

list: more memory is needed
      homogenous memory in numpy results in more speed  -> different data types
      c_array f_array  ---->  which operations are slow which are fast? which axes?
      default is memory view in numpy -----> no copy
      debug ---> flags, strides, data, (under the hood information)
      
      vectorization --> beutiy, no foor loop, formula == code
      broadcasting --> no broadcasting in python